Using Yolov5 Model

In [2]:
from ultralytics import YOLO
import cv2

# Load the YOLOv5 model
model = YOLO('yolov5s.pt')  # YOLOv5s is a small pre-trained model

# Open a video file or real-time camera
cap = cv2.VideoCapture(0)  # Use 0 for webcam or provide a video path
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model.predict(source=frame, save=False)

    # Draw bounding boxes
    annotated_frame = results[0].plot()

    # Display the frame
    cv2.imshow('YOLOv5 Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()



PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


0: 480x640 1 person, 147.1ms
Speed: 0.0ms preprocess, 147.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 145.6ms
Speed: 0.0ms preprocess, 145.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 125.7ms
Speed: 0.0ms preprocess, 125.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 117.0ms
Speed: 0.0ms preprocess, 117.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 115.7ms
Speed: 0.0ms preprocess, 115.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 couch, 118.1ms
Speed: 2.3ms preprocess, 118.1ms inference, 0.0ms postprocess per

Using SSD Model

In [1]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained SSD MobileNet model from TensorFlow Hub
ssd_model = tf.saved_model.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
category_index = {1: "Person", 2: "Bicycle", 3: "Car", 4: "Motorcycle", 5: "Airplane", 6: "Bus", 7: "Train", 8: "Truck", 9: "Boat", 10: "Traffic Light"}

# Function to detect objects
def detect_objects(frame, model):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model(input_tensor)

    # Extract information from detections
    detection_boxes = detections["detection_boxes"][0].numpy()
    detection_scores = detections["detection_scores"][0].numpy()
    detection_classes = detections["detection_classes"][0].numpy().astype(np.int32)

    return detection_boxes, detection_scores, detection_classes

# Function to draw bounding boxes
def draw_boxes(frame, boxes, scores, classes, threshold=0.5):
    h, w, _ = frame.shape
    for i, box in enumerate(boxes):
        if scores[i] > threshold:
            y1, x1, y2, x2 = box
            x1, x2, y1, y2 = int(x1 * w), int(x2 * w), int(y1 * h), int(y2 * h)
            class_name = category_index.get(classes[i], "Unknown")
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name}: {scores[i]:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

# Open video capture (real-time or pre-recorded video)
video_path = 0  # Use 0 for webcam or replace with video file path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize and normalize the frame
    input_frame = cv2.resize(frame, (300, 300))
    input_frame = np.array(input_frame) / 255.0

    # Detect objects
    boxes, scores, classes = detect_objects(input_frame, ssd_model)

    # Draw boxes on the original frame
    output_frame = draw_boxes(frame, boxes, scores, classes)

    # Display the output
    cv2.imshow("SSD Object Detection", output_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


UnimplementedError: File system scheme 'https' not implemented (file: 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2/saved_model.pb')